# Loading Data

## QUOKKA Dataset

This section details the integration of the `QUOKKA` dataset with the yt framework, following a style similar to the [AMReX/BoxLib Data](https://yt-project.org/doc/examining/loading_data.html#amrex-boxlib-data) documentation.

### Overview

The `QUOKKA` dataset is a simulation output based on the AMReX/BoxLib framework, analogous to Orion, Nyx, Maestro, Castro, IAMR, and WarpX. This yt frontend, developed by [Chongchong](https://github.com/chongchonghe) and [Rongjun](https://github.com/Rongjun-ANU), enables seamless integration of QUOKKA data into yt's analysis and visualization tools.

### Features

- **Header Management**: Automatically detects and loads six mandatory native gas fields (density, energy, internal energy, and x/y/z-momentum) along with optional fields (temperature, scalar, radiation, and placeholder magnetic fields).
- **Units Management**: Maps native field names (e.g., `('boxlib', 'gasDensity')`) to derived field tuples (e.g., `('gas', 'density')`) and assigns appropriate physical units.
- **Metadata Management**: Parses the `Header` file and `metadata.yaml` to extract simulation parameters and field information.
- **Particle Support**: Detects particle types and fields from `*_particles` directories, including support for custom particle field units as defined in `Fields.yaml`.

### Data Structure

A standard QUOKKA dataset contains the following components:

```bash
dataset_folder/
├── Level_0/         # QUOKKA simulation data
├── Header           # Dataset header information
└── metadata.yaml    # Configuration file for yt integration
```

For datasets with particles, the structure is extended as follows:

```bash
dataset_folder/
├── Level_0/
├── Header
├── metadata.yaml
├── XXX_particles/     # Particle data directory
│   ├── Fields.yaml    # Particle field names and units
│   ├── Header
│   └── Level_0/
```

### Field Parsing

QUOKKA datasets always include six mandatory native gas fields:

```python
[('boxlib', 'gasDensity'),
 ('boxlib', 'gasEnergy'),
 ('boxlib', 'gasInternalEnergy'),
 ('boxlib', 'x-GasMomentum'),
 ('boxlib', 'y-GasMomentum'),
 ('boxlib', 'z-GasMomentum')]
```

These fields are mapped to derived fields with physical units:

```python
[('gas', 'density'),
 ('gas', 'energy'),
 ('gas', 'internalEnergy'),
 ('gas', 'x-momentum'),
 ('gas', 'y-momentum'),
 ('gas', 'z-momentum')]
```

For radiation, native fields such as:

```python
[('boxlib', 'radEnergy-Group0'),
 ('boxlib', 'x-RadFlux-Group0'),
 ('boxlib', 'y-RadFlux-Group0'),
 ('boxlib', 'z-RadFlux-Group0')]
```

are converted to derived fields with units:

```python
[('rad', 'energy_density_0'),
 ('rad', 'flux_density_x_0'),
 ('rad', 'flux_density_y_0'),
 ('rad', 'flux_density_z_0')]
```

Since the native `boxlib` fields are dimensionless, the conversion assigns physical units (e.g., `g/cm**3` for density) to facilitate compatibility with other Python packages built on yt.

### Loading a QUOKKA Dataset

To load a QUOKKA dataset, simply use the standard yt load function:

```python
ds = yt.load("plt007")
```

Verify that the dataset has been recognized as a QUOKKA dataset:

```python
print(ds.parameters['HydroMethod'])  # Should output 'Quokka'
```

### Metadata and Header Parsing

All details from the dataset's `Header` and `metadata.yaml` files are accessible via:

```python
print(ds.parameters)
```

### Visualization Examples

Plot gas density on the x-y plane:

```python
import yt
yt.SlicePlot(ds, 'z', ('gas', 'density'))
```

For radiation fields:

```python
yt.SlicePlot(ds, 'z', ('rad', 'energy_density_0'))
```

To visualize particles:

```python
p = yt.SlicePlot(ds, 'z', ('rad', 'energy_density_0'), center='c')
p.annotate_particles(1, p_size=400., col='blue', marker='*', ptype='Rad_particles')
p
```

### Usage

Load and visualize a QUOKKA dataset as follows:

```python
import yt
ds = yt.load("path_to_quokka_dataset")
ad = ds.all_data()
print(ad[('gas', 'density')])
yt.SlicePlot(ds, 'z', ('gas', 'density'))
```

For debugging:

```python
yt.set_log_level("DEBUG")
```


In [ ]:
# Example code for loading and visualizing a QUOKKA dataset
import yt

# Load the dataset
ds = yt.load("path_to_quokka_dataset")

# Verify the dataset type
print("HydroMethod:", ds.parameters.get('HydroMethod', 'Unknown'))

# Create a slice plot of gas density
slc = yt.SlicePlot(ds, 'z', ('gas', 'density'))
slc.show()
